In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## define a net-- LeNet-5

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 3, 3])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 576])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [4]:
params = list(net.parameters())
params

[Parameter containing:
 tensor([[[[-0.3205, -0.1745,  0.1085],
           [ 0.2516, -0.2107, -0.0206],
           [ 0.2595,  0.2865, -0.2976]]],
 
 
         [[[ 0.2556, -0.2376,  0.1733],
           [-0.1275, -0.0442, -0.3245],
           [-0.2551, -0.2797, -0.3171]]],
 
 
         [[[-0.2548,  0.3278,  0.0873],
           [ 0.0344,  0.0687, -0.2093],
           [ 0.0934, -0.3255, -0.2809]]],
 
 
         [[[ 0.0131, -0.2448,  0.0323],
           [-0.0572,  0.0250,  0.0201],
           [-0.2382,  0.0284,  0.2721]]],
 
 
         [[[-0.0984, -0.2502, -0.2742],
           [-0.1103,  0.1118,  0.0399],
           [-0.2475, -0.1893,  0.0051]]],
 
 
         [[[-0.2054, -0.2294,  0.0654],
           [ 0.0239, -0.1071,  0.0255],
           [-0.0897,  0.0576,  0.0373]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.2785,  0.3038, -0.2244, -0.0777,  0.1916,  0.2181],
        requires_grad=True),
 Parameter containing:
 tensor([[[[-0.0971,  0.0744, -0.0232],
           [-0.0112,  0.

In [5]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.1205, -0.0728,  0.1330,  0.0376,  0.0162,  0.0004,  0.0938,  0.0316,
          0.0281,  0.0968]], grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()

In [7]:
out.backward(torch.rand(1,10))

## loss function

In [8]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2047, grad_fn=<MseLossBackward>)


In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [10]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0103, -0.0104,  0.0141, -0.0144, -0.0025, -0.0217])


## optimizer

In [11]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update